In [4]:
! pip3 install elasticsearch --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [21]:
import pandas as pd

In [62]:
# create documents for elasticsearch
df = pd.read_csv("../../podcasts-no-audio-13GB-selected/metadata_with_episode_dates_and_category.tsv",sep='\t')

In [26]:
from elasticsearch import Elasticsearch, helpers
INDEX_NAME = "search-spotify-dataset-visualizer"
#with open("spotify-dataset-visualization.csv", "r") as fp:
#    key = fp.readlines()

client = Elasticsearch(
"https://527db13b131f4f1b90590b6f6c0039f6.us-central1.gcp.cloud.es.io:443",
  # api_key=key[0]
  # api_key="UDFJMVBJd0JRcERJVVMzblQ1RDA6UE1KSnNjbGhUdnU0dGlJR3RzaEdWZw=="
  api_key="STFJVlc0d0JRcERJVVMzblRzTG86OHBDcEUzLTNUeHVENUdISmxmRlZ2dw=="
)

client.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': '527db13b131f4f1b90590b6f6c0039f6', 'cluster_uuid': 'c5RsPX1ZRNeYPFZkw-JhLQ', 'version': {'number': '8.11.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6f9ff581fbcde658e6f69d6ce03050f060d1fd0c', 'build_date': '2023-11-11T10:05:59.421038163Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
df[df['episode_filename_prefix']=='1S9X55ZcDVk5AgA2nG0yuz']

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,release_date,first_dir,second_dir,category,mean_show_duration,top_5
19696,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,spotify:episode:1S9X55ZcDVk5AgA2nG0yuz,Love Invocation 03: Love Transforms,What’s good witches? Welcome to day 3 of our L...,5.72255,show_7Kf4MwDBFH2QXaLZadac7W,1S9X55ZcDVk5AgA2nG0yuz,2019-06-04,7,K,Religion & Spirituality,7.365535,False


In [151]:
df.category.iloc[:10]

0                                 Leisure
1                              True Crime
2                           Sports,Soccer
3                                  Sports
4                              Arts,Books
5                               Education
6              Health & Fitness,Sexuality
7    Religion & Spirituality,Christianity
8                              Government
9              Education,Self-Improvement
Name: category, dtype: object

## Adding Topics Chart 

In [54]:
# adding topics
import pickle
with open('all_inferenced_topics.pkl', 'rb') as fp:
    episode_topics = pickle.load(fp)

df_topics = pd.DataFrame(episode_topics)
df_topics.columns = ['episode_filename_prefix', 'topics']
df_topics['episode_filename_prefix'] = df_topics['episode_filename_prefix'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [55]:
df = pd.merge(df_topics, df, on='episode_filename_prefix', how='inner')


In [58]:
from collections import Counter

def string_histogram(string_list):
    counts = Counter(string_list)
    return counts

df['topics_count'] = df['topics'].apply(string_histogram)

agg_df = df.groupby('show_uri')['topics_count'].sum().reset_index()

# merged_df = df.merge(agg_df, on='show_uri', suffixes=('', '_group'), how='inner')

# merged_df

merged_df = df.merge(agg_df, on='show_uri', how='left')

merged_df.rename(columns={'topics_count_x': 'topics_count_episode', 'topics_count_y': 'topics_count_show'}, inplace=True)

df = merged_df




In [68]:
# df.drop(['topics_x'], axis=1, inplace=True)
# df.drop(['topics_y'], axis=1, inplace=True)

In [75]:
import pandas as pd
import altair as alt
from collections import Counter
import json

charts = []

def generate_topics_show_chart(counter):
    keys = list(counter.keys())
    values = list(counter.values())
    
    chart = alt.Chart(pd.DataFrame({'Key': keys, 'Value': values})).mark_bar().encode(
        x=alt.X('Key:N', title='Key'),
        y=alt.Y('Value:Q', title='Value')
    ).properties(
        title=f'Show Topics Count'
    )

    return json.dumps(chart.to_dict())

df['topics_count_show_chart'] = df['topics_count_show'].apply(generate_topics_show_chart)


## Add duration of episodes

In [194]:
df.shape

(105360, 18)

In [216]:
import json
with open("../visualization_data/show/show.json", "r") as fp:
    data = json.load(fp)

# each element of show_list has a show_name: chart to visualize.
show_list = [{k:v['chart2']}for k,v in data.items()]
df['episode_duration_by_show'] = ""

for show in show_list:
    show_name = list(show.keys())[0]
    df.loc[df['show_name'] == show_name, 'episode_duration_by_show'] = json.dumps(show[show_name])

In [217]:
df.head()

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,release_date,first_dir,second_dir,category,mean_show_duration,top_5,episode_duration_by_show
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,2019-12-18,2,N,Leisure,37.729706,False,
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA,2020-01-30,1,5,True Crime,7.290034,False,"{""config"": {""view"": {""continuousWidth"": 300, ""..."
2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,['en'],https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,2019-01-18,6,v,"Sports,Soccer",55.862658,False,"{""config"": {""view"": {""continuousWidth"": 300, ""..."
3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,['en-US'],https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,2019-03-01,5,B,Sports,44.14528,True,"{""config"": {""view"": {""continuousWidth"": 300, ""..."
4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,['en'],https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,,7,w,"Arts,Books",42.84763,False,"{""config"": {""view"": {""continuousWidth"": 300, ""..."


## Add number of total_show_episodes to the dataframe 

In [115]:
df_total_show_episodes = pd.DataFrame(df.groupby('show_name').size().sort_values()).reset_index()
df_total_show_episodes.columns=['show_name','total_show_episodes']
df_total_show_episodes

,show_name,total_show_episodes
0,,1
1,Produce Like a Boss - with Kris Bradley,1
2,Product Manager Asia Podcast: Interviews with ...,1
3,Productive & Peaceful,1
4,Professional Carpet Cleaners and Restorers,1
...,...,...
18285,Daily Fortnite,417
18286,Optimal Living Daily: Personal Development & M...,424
18287,Coach Corey Wayne,615
18288,Dr Berg’s Healthy Keto and Intermittent Fastin...,746


In [124]:
df2 = df.merge(df_total_show_episodes,on='show_name',how='left')
df2 = pd.DataFrame(df2[['total_show_episodes','episode_uri','release_date','episode_filename_prefix', 'show_uri', 'show_name', 'show_description', 'publisher', 'language', 'rss_link', 'episode_name', 'episode_description', 'duration', 'show_filename_prefix']])
df2['topics'] = ""
df2['topics_count_show_chart'] = ""

In [125]:
df2[df['show_name']=='BOSSY Bruja Prayers']

,total_show_episodes,episode_uri,release_date,episode_filename_prefix,show_uri,show_name,show_description,publisher,language,rss_link,episode_name,episode_description,duration,show_filename_prefix,topics,topics_count_show_chart
17452,14,spotify:episode:1HmdZPNVswDhWvGevw5Z5j,2019-06-11,1HmdZPNVswDhWvGevw5Z5j,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 10: Love Persists,Welcome to day 10 of our love invocation sweet...,4.299950,show_7Kf4MwDBFH2QXaLZadac7W,,
19696,14,spotify:episode:1S9X55ZcDVk5AgA2nG0yuz,2019-06-04,1S9X55ZcDVk5AgA2nG0yuz,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 03: Love Transforms,What’s good witches? Welcome to day 3 of our L...,5.722550,show_7Kf4MwDBFH2QXaLZadac7W,,
24698,14,spotify:episode:1ofoG7xOVfo6WMedf7xcYH,2019-06-07,1ofoG7xOVfo6WMedf7xcYH,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 06: Love Cares,What’s good witches! It’s day 6 of our LOVE IN...,5.547633,show_7Kf4MwDBFH2QXaLZadac7W,,
26498,14,spotify:episode:1xAOzDOdOWSGBY6jyyjZQO,2019-06-12,1xAOzDOdOWSGBY6jyyjZQO,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 12: Love Fortifies,What’s good witches? Welcome to day 12 of our ...,4.087867,show_7Kf4MwDBFH2QXaLZadac7W,,
48465,14,spotify:episode:3YGFZLBXRw8kJs7wXp6xYn,2019-06-01,3YGFZLBXRw8kJs7wXp6xYn,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 01: Love Heals,"Peace and blessings, I am Ayodele Fuega of bos...",6.685800,show_7Kf4MwDBFH2QXaLZadac7W,,
59588,14,spotify:episode:4NRfWHiXpQdMK4Cvk8GzPJ,2019-06-03,4NRfWHiXpQdMK4Cvk8GzPJ,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 02: Love is Patient,"Prayer God, this prayer is a little bit diffi...",6.602983,show_7Kf4MwDBFH2QXaLZadac7W,,
60762,14,spotify:episode:4SvMPttDl7WqXIuNVxTrXj,2019-06-09,4SvMPttDl7WqXIuNVxTrXj,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 08: Love Remains,What’s good witches! Welcome to day 8 of our l...,4.649417,show_7Kf4MwDBFH2QXaLZadac7W,,
62952,14,spotify:episode:4d6bxJ1FtMbfo2y5kQmIZw,2019-07-19,4d6bxJ1FtMbfo2y5kQmIZw,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,LOVE INVOCATION 13: LOVE NEVER FAILS,"Day 13: LOVE ALWAYS TRUSTS, ALWAYS HOPES, ALWA...",4.890900,show_7Kf4MwDBFH2QXaLZadac7W,,
71272,14,spotify:episode:5FGq3Nxa6hXBcBLpq9DbL4,2019-06-11,5FGq3Nxa6hXBcBLpq9DbL4,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 11: Love Reincarnates,D A Y 11: L O V E R E I N C A R N A T E S Wha...,4.807300,show_7Kf4MwDBFH2QXaLZadac7W,,
76091,14,spotify:episode:5bc22mmHX7yLd91NEwIQbV,2019-06-11,5bc22mmHX7yLd91NEwIQbV,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,Love Invocation 09: Love Enchants,What’s good witches! Today is day 9 of our pro...,4.037567,show_7Kf4MwDBFH2QXaLZadac7W,,


## ES Update Example
Push new objects to ES

In [158]:
df.categories.fillna('')

(105360, 16)

In [159]:
df.shape

(105360, 18)

In [218]:
# prepare bulk data
actions = [
    {
        "_op_type": "update", # change to "update" when updating existing documents
        "_index": INDEX_NAME,
        "_id": row['episode_uri'],
        "doc": {"doc":{'episode_duration_by_show':row['episode_duration_by_show']}},
        "doc_as_upsert": True
                
    }
    for index, row in df.iterrows()
]

In [157]:
actions[0]

{'_op_type': 'update',
 '_index': 'search-spotify-dataset-visualizer',
 '_id': 'spotify:episode:000A9sRBYdVh66csG2qEdj',
 'doc': {'doc': {'category': 'Leisure'}},
 'doc_as_upsert': True}

In [219]:
# bulk operation on the documents
from elasticsearch.helpers import BulkIndexError

try:
    helpers.bulk(client, actions)
except BulkIndexError as e:
    print("An error occurred during bulk updates:")
    print(e)

    # Optionally, print more detailed information about each error
    for error in e.errors:
        print(error)

In [172]:
df.iloc[1]

show_uri                                 spotify:show:15iWCbU7QoO23EndPEO6aN
show_name                                              Morning Cup Of Murder
show_description           Ever wonder what murder took place on today in...
publisher                                              Morning Cup Of Murder
language                                                              ['en']
rss_link                             https://anchor.fm/s/b07181c/podcast/rss
episode_uri                           spotify:episode:000HP8n3hNIfglT2wSI2cA
episode_name               The Goleta Postal Facility shootings- January ...
episode_description        See something, say something. It’s a mantra ma...
duration                                                            6.019383
show_filename_prefix                             show_15iWCbU7QoO23EndPEO6aN
episode_filename_prefix                               000HP8n3hNIfglT2wSI2cA
release_date                                                      2020-01-30

In [169]:
client.update(index="search-spotify-dataset-visualizer", id="spotify:episode:000A9sRBYdVh66csG2qEdj", doc={'doc':{'category':'Fun'}})

ObjectApiResponse({'_index': 'search-spotify-dataset-visualizer', '_id': 'spotify:episode:000A9sRBYdVh66csG2qEdj', '_version': 8, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 422941, '_primary_term': 1})

In [173]:
d = client.get(index="search-spotify-dataset-visualizer", id="spotify:episode:000HP8n3hNIfglT2wSI2cA")
d['_source']['doc']

{'episode_filename_prefix': '000HP8n3hNIfglT2wSI2cA',
 'topics': '',
 'show_uri': 'spotify:show:15iWCbU7QoO23EndPEO6aN',
 'show_name': 'Morning Cup Of Murder',
 'show_description': 'Ever wonder what murder took place on today in true crime history? If so, sit back and grab a cup of coffee as you enjoy your daily dose of TC goodness. Your host, Korina Biemesderfer, guides you through history with tales of murder, abduction, serial killers, crimes of passion, cults and more in this short form daily true crime podcast. Support this podcast: https://anchor.fm/morning-cup-of-murder/support',
 'publisher': 'Morning Cup Of Murder',
 'language': "['en']",
 'rss_link': 'https://anchor.fm/s/b07181c/podcast/rss',
 'episode_name': 'The Goleta Postal Facility shootings- January 30 2020 - Today in True Crime History',
 'episode_description': 'See something, say something. It’s a mantra many live by. If you see something strange, call it in or make someone aware, even if it seems innocuous. Jennifer 

In [174]:
import pprint
pprint.pprint(dict(client.get(index="search-spotify-dataset-visualizer", id="spotify:episode:000HP8n3hNIfglT2wSI2cA")))

{'_id': 'spotify:episode:000HP8n3hNIfglT2wSI2cA',
 '_index': 'search-spotify-dataset-visualizer',
 '_primary_term': 1,
 '_seq_no': 421942,
 '_source': {'doc': {'category': 'True Crime',
                     'duration': 6.019383333333334,
                     'episode_description': 'See something, say something. '
                                            'It’s a mantra many live by. If '
                                            'you see something strange, call '
                                            'it in or make someone aware, even '
                                            'if it seems innocuous. Jennifer '
                                            'San Marco had strange behaviors. '
                                            'It was clear to many that the '
                                            'woman suffered from mental '
                                            'illness that was being untreated. '
                                            'But, many w

In [122]:
d = dict(client.get(index="search-spotify-dataset-visualizer", id="spotify:episode:1S9X55ZcDVk5AgA2nG0yuz"))
print(d.keys())
print(d['_source'].keys())
d['_source']['doc'].keys()

dict_keys(['_index', '_id', '_version', '_seq_no', '_primary_term', 'found', '_source'])
dict_keys(['doc', 'episode_name', 'total_show_episodes', 'total_show_episodes_v2'])


dict_keys(['episode_filename_prefix', 'topics', 'show_uri', 'show_name', 'show_description', 'publisher', 'language', 'rss_link', 'episode_name', 'episode_description', 'duration', 'show_filename_prefix', 'topics_count_show_chart'])

## Query / Index  Examples

In [84]:
# deletes all records in elasticsearch
client.delete_by_query(index=[INDEX_NAME], body={"query": {"match_all": {}}})

ObjectApiResponse({'took': 5571, 'timed_out': False, 'total': 105360, 'deleted': 105360, 'batches': 106, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [189]:
df = df.fillna("")

In [78]:
# df.drop(['topics_count_episode', 'topics_count_show'], axis=1, inplace=True)

In [81]:
df['topics_count_show_chart'].iloc[0]

'{"config": {"view": {"continuousWidth": 300, "continuousHeight": 300}}, "data": {"name": "data-ab57db7ce3afd3c12a486e050db8c4a4"}, "mark": {"type": "bar"}, "encoding": {"x": {"field": "Key", "title": "Key", "type": "nominal"}, "y": {"field": "Value", "title": "Value", "type": "quantitative"}}, "title": "Show Topics Count", "$schema": "https://vega.github.io/schema/vega-lite/v5.15.1.json", "datasets": {"data-ab57db7ce3afd3c12a486e050db8c4a4": [{"Key": "diaries_&_daily_life", "Value": 14}, {"Key": "relationships", "Value": 6}, {"Key": "other_hobbies", "Value": 1}]}}'

In [126]:
# prepare bulk data
actions = [
    {
        "_op_type": "index", # change to "update" when updating existing documents
        "_index": INDEX_NAME,
        "_id": row['episode_uri'],
        "doc": row.drop('episode_uri').drop('release_date').to_dict()
    }
    for index, row in df2.iterrows()
]

In [87]:
actions[0]

{'_op_type': 'index',
 '_index': 'search-spotify-dataset-visualizer',
 '_id': 'spotify:episode:1S9X55ZcDVk5AgA2nG0yuz',
 'doc': {'episode_filename_prefix': '1S9X55ZcDVk5AgA2nG0yuz',
  'topics': ['diaries_&_daily_life', 'relationships'],
  'show_uri': 'spotify:show:7Kf4MwDBFH2QXaLZadac7W',
  'show_name': 'BOSSY Bruja Prayers',
  'show_description': 'Our prayer line has a new home! This channel will be the home of BOSSYbruja’s prayer invocations and guided meditations. I pray you enjoy! Be sure to check out our Podcast “BOSSY Bruja Podcast!” Enjoy.  Support this podcast: https://anchor.fm/bossybrujaprayers/support',
  'publisher': 'Ayodele Fuega',
  'language': "['en']",
  'rss_link': 'https://anchor.fm/s/990caa0/podcast/rss',
  'episode_name': 'Love Invocation 03: Love Transforms',
  'episode_description': 'What’s good witches? Welcome to day 3 of our LOVE INVOCATION. So, y’all know I be sitting around waiting for Spirit to tell me what the topic is, right? [Laughter] Today’s topic is L

In [88]:
# bulk operation on the documents
from elasticsearch.helpers import BulkIndexError

try:
    helpers.bulk(client, actions)
except BulkIndexError as e:
    print("An error occurred during bulk indexing:")
    print(e)

    # Optionally, print more detailed information about each error
    for error in e.errors:
        print(error)

In [60]:
client.get(index="search-spotify-dataset-visualizer", id="spotify:episode:1S9X55ZcDVk5AgA2nG0yuz")

ObjectApiResponse({'_index': 'search-spotify-dataset-visualizer', '_id': 'spotify:episode:1S9X55ZcDVk5AgA2nG0yuz', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'doc': {'episode_filename_prefix': '1S9X55ZcDVk5AgA2nG0yuz', 'topics': ['diaries_&_daily_life', 'relationships'], 'show_uri': 'spotify:show:7Kf4MwDBFH2QXaLZadac7W', 'show_name': 'BOSSY Bruja Prayers', 'show_description': 'Our prayer line has a new home! This channel will be the home of BOSSYbruja’s prayer invocations and guided meditations. I pray you enjoy! Be sure to check out our Podcast “BOSSY Bruja Podcast!” Enjoy.  Support this podcast: https://anchor.fm/bossybrujaprayers/support', 'publisher': 'Ayodele Fuega', 'language': "['en']", 'rss_link': 'https://anchor.fm/s/990caa0/podcast/rss', 'episode_name': 'Love Invocation 03: Love Transforms', 'episode_description': 'What’s good witches? Welcome to day 3 of our LOVE INVOCATION. So, y’all know I be sitting around waiting for Spirit to tell me wha

In [19]:
client.search(index=INDEX_NAME, q="inside")

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.190946, 'hits': [{'_index': 'search-spotify-dataset-visualizer', '_id': 'spotify:episode:001UfOruzkA3Bn1SPjcdfa', '_score': 1.190946, '_source': {'doc': {'show_uri': 'spotify:show:6vZRgUFTYwbAA79UNCADr4', 'show_name': 'Inside The 18 : A Podcast for Goalkeepers by Goalkeepers', 'show_description': 'Inside the 18 is your source for all things Goalkeeping! Each week we are joined by guests from around the world. We\xa0recap the weeks events,\xa0discuss new training techniques\xa0and have candid conversations with professional goalkeepers and goalkeeper coaches. The show is a must listen for the goalkeeping enthusiast!    #insidethe18 #goalkeeperpodcast #thegoalkeepers #goalkicks #ederson #NWSL #ashlynharris #UWSNT #MLS #USMNT #goalkeepercoaches #degea #neuer #navas #areola #oblak #USL #goalkeepersaves #court

In [ ]:
client.q

In [4]:
query = {
    "query": {
        "query_string": {
            "query": "show_name:Basketball Theory"
        }
    }
}

response = client.search(index=INDEX_NAME, body=query)

In [7]:
import pprint
pprint.pprint(dict(response))

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'spotify:episode:3mE5JKsWS76y8HRiluTjGa',
                    '_index': 'search-spotify-dataset-visualizer',
                    '_score': 10.964263,
                    '_source': {'doc': {'duration': 6.0186166666666665,
                                        'episode_description': 'Bowlby’s '
                                                               'theory and how '
                                                               'to apply '
                                                               'attachment '
                                                               'theory to EY '
                                                               'practice  ',
                                        'episode_filename_prefix': '3mE5JKsWS76y8HRiluTjGa',
                                        'episode_name': 'HB Attachment Theory '
                                            

In [19]:
dict(response)['hits']['hits'][0]['_source']['doc'].keys()

dict_keys(['episode_filename_prefix', 'topics', 'show_uri', 'show_name', 'show_description', 'publisher', 'language', 'rss_link', 'episode_name', 'episode_description', 'duration', 'show_filename_prefix', 'topics_count_show_chart'])

In [11]:

print(f"{len(dict(response)['hits'])} hits found  ")

{'total': {'value': 517, 'relation': 'eq'},
 'max_score': 10.964263,
 'hits': [{'_index': 'search-spotify-dataset-visualizer',
   '_id': 'spotify:episode:3mE5JKsWS76y8HRiluTjGa',
   '_score': 10.964263,
   '_source': {'doc': {'episode_filename_prefix': '3mE5JKsWS76y8HRiluTjGa',
     'topics': ['diaries_&_daily_life', 'family'],
     'show_uri': 'spotify:show:2Yxd1NPGSSxPv0gBgR8wtu',
     'show_name': 'HB Attachment theory - Part 1. Mary Ainsworth ',
     'show_description': 'Attachment theory discussion',
     'publisher': 'Attachment Avenue HB ',
     'language': "['en']",
     'rss_link': 'https://anchor.fm/s/8dc9760/podcast/rss',
     'episode_name': 'HB Attachment Theory - Part 2. John Bowlby ',
     'episode_description': 'Bowlby’s theory and how to apply attachment theory to EY practice  ',
     'duration': 6.0186166666666665,
     'show_filename_prefix': 'show_2Yxd1NPGSSxPv0gBgR8wtu',
     'topics_count_show_chart': '{"config": {"view": {"continuousWidth": 300, "continuousHeight